In [1]:
import numpy as np
import json
import torch
from datasets import Dataset
import random
from transformers import AutoTokenizer, AutoModel

from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import evaluate
from sklearn.metrics import f1_score
import tqdm

from evaluate import load
import numpy as np

# Create Datasets

In [2]:
# laod questions
questions_dict = {}
with open("questions.jl", "r", encoding="utf-8") as file:
    for line in file:
        row = json.loads(line)
        questions_dict[row['_id']] = row['text']
print("Questions:")
print(next(iter(questions_dict.items())))

Questions:
('1', 'Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega karze pozbawienia wolności?')


In [3]:
# laod answears
answears_dict = {}
with open("answers.jl", "r", encoding="utf-8") as file:
    for line in file:
        row = json.loads(line)
        answears_dict[row['question-id']] = row['answer']
print("answers:")
print(next(iter(answears_dict.items())))

answers:
('1', 'Tak, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3.')


In [4]:
# laod passages and titles
passages_dict = {}
with open("passages.jl", "r", encoding="utf-8") as file:
    for line in file:
        row = json.loads(line)
        passages_dict[row['_id']] = row['text']
print("Passages (context):")
print(next(iter(passages_dict.items())))

Passages (context):
('2004_2387_1', 'Art. 1. Wyraża się zgodę na dokonanie przez Prezydenta Rzeczypospolitej Polskiej ratyfikacji Konwencji o pozbawianiu uprawnień do kierowania pojazdami, sporządzonej w Luksemburgu dnia 17 czerwca 1998 r.')


In [5]:
# Create datasets:
# train dataset (Poquad)

id_ = 0
train_data = []
with open('poquad-train.json', "r", encoding="utf-8") as file:
        data = json.load(file)
        for line in data['data']:
            for paragraph in line['paragraphs']:
                context = paragraph['context']
                for qa in paragraph['qas']:
                    if not(qa['is_impossible']):
                        question = qa['question']
                        for ans in qa['answers']:
                            answear = {"text": [ans['generative_answer']]}
                            train_data.append({
                                "id" : id_,
                                "context": context,
                                "question": question,
                                "answers": answear,
                            })
                            id_ += 1
train_dataset = Dataset.from_list(train_data)

In [6]:
# Validation dataset (Poquad)

id_ = 0
val_data = []
with open('poquad-dev.json', "r", encoding="utf-8") as file:
        data = json.load(file)
        for line in data['data']:
            for paragraph in line['paragraphs']:
                context = paragraph['context']
                for qa in paragraph['qas']:
                    if not(qa['is_impossible']):
                        question = qa['question']
                        for ans in qa['answers']:
                            answear = {"text": [ans['generative_answer']]}
                            val_data.append({
                                "id" : id_,
                                "context": context,
                                "question": question,
                                "answers": answear,
                            })
                            id_ += 1
val_dataset = Dataset.from_list(val_data)

In [7]:
val_dataset[0]

{'id': 0,
 'context': 'Pisma rabiniczne – w tym Miszna – stanowią kompilację poglądów różnych rabinów na określony temat. Zgodnie z wierzeniami judaizmu Mojżesz otrzymał od Boga całą Torę, ale w dwóch częściach: jedną część w formie pisanej, a drugą część w formie ustnej. Miszna – jako Tora ustna – była traktowana nie tylko jako uzupełnienie Tory spisanej, ale również jako jej interpretacja i wyjaśnienie w konkretnych sytuacjach życiowych. Tym samym Miszna stanowiąca kodeks Prawa religijnego zaczęła równocześnie służyć za jego ustnie przekazywany podręcznik.',
 'question': 'Czym są pisma rabiniczne?',
 'answers': {'text': ['kompilacją poglądów różnych rabinów na określony temat']}}

In [8]:
# test dataset (Legal questions)
test_dataset = []
with open("relevant.jl", "r", encoding="utf-8") as file:
    for line in file:
        row = json.loads(line)
        # take only those question that have answers and answear is not empty
        if row['question-id'] not in answears_dict or len(answears_dict[row['question-id']]) == 0:
            continue
        test_dataset.append(
            {
                "id" : row['question-id'],
                #'title': title_dict[row['passage-id']],
                'context': passages_dict[row['passage-id']],
                'question': questions_dict[row['question-id']],
                'answers': {"text": [answears_dict[row['question-id']]]}
            }
        )
test_dataset = Dataset.from_list(test_dataset)
test_dataset[0]

{'id': '1',
 'context': 'Art. 345. § 1. Żołnierz, który dopuszcza się czynnej napaści na przełożonego, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3. § 2. Jeżeli sprawca dopuszcza się czynnej napaści w związku z pełnieniem przez przełożonego obowiązków służbowych albo wspólnie z innymi żołnierzami lub w obecności zebranych żołnierzy, podlega karze pozbawienia wolności od 6 miesięcy do lat 8. § 3. Jeżeli sprawca czynu określonego w § 1 lub 2 używa broni, noża lub innego podobnie niebezpiecznego przedmiotu, podlega karze pozbawienia wolności od roku do lat 10. § 4. Karze przewidzianej w § 3 podlega sprawca czynu określonego w § 1 lub 2, jeżeli jego następstwem jest skutek określony w art. 156 lub 157 § 1.',
 'question': 'Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega karze pozbawienia wolności?',
 'answers': {'text': ['Tak, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3.']}}

In [9]:
print("Train set size:", train_dataset.num_rows)
print("Val set size:", val_dataset.num_rows)
print("Test set size:", test_dataset.num_rows)

Train set size: 46187
Val set size: 5764
Test set size: 593


# Model - T5 allegro - Training

In [10]:
tokenizer = AutoTokenizer.from_pretrained("allegro/plt5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("allegro/plt5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [11]:
def preprocess_data(examples):
    # all inputs should have the same format. Pytanie + context in string.
    inputs = ["pytanie: " + q + "  context: " + c for q, c in zip(examples["question"], examples["context"])]
    targets = [ans["text"][0] for ans in examples["answers"]]
    # Tokenizing inputs and targets
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        padding="max_length",
    )
    # Tokenizing targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=128,
            padding="max_length",
        )
    # Replace padding token id for labels to ignore index (-100) in loss computation
    labels["input_ids"] = [
        [l if l != tokenizer.pad_token_id else -100 for l in label]
        for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [12]:
# Tokenize all data:
train_dataset = train_dataset.map(preprocess_data, batched=True)
val_dataset = train_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/46187 [00:00<?, ? examples/s]

/opt/anaconda3/envs/NLP/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/46187 [00:00<?, ? examples/s]

Map:   0%|          | 0/593 [00:00<?, ? examples/s]

# Fine tuning polish T5 for - Abstractive Question Answearing

Commented as it did not work on my pc (not enough resources, google collab refused to work :( but script works!)

In [13]:
# training_args = TrainingArguments(
#     output_dir="./t5_qa_model",
#     eval_strategy="steps",
#     learning_rate=3e-5,
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=1,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     save_total_limit=1,
#     logging_dir="./logs",
#     logging_steps=500,
#     save_steps=1000,
#     report_to="none",
#     disable_tqdm=False,
# )

In [14]:
# if torch.backends.mps.is_available():
#     print("Torch MPS availavle")
#     device = torch.device("mps")
# else:
#     print("running on CPU")
#     device = torch.device("cpu")

In [15]:
# model.to(device)

In [16]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer,
# )

In [17]:
# trainer.train()

In [18]:
#trainer.save_model("./t5_qa_model_saved")

# Loading Polish T5 model fine-tuned on poquad dataset

In [19]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("apohllo/plt5-base-poquad")
model = AutoModelForSeq2SeqLM.from_pretrained("apohllo/plt5-base-poquad")

In [20]:
# Tokenize all data again to be sure it is tokenized with correct tokenizer and ready to model.
# It should be the same models but to ensure myself:


# train_dataset = train_dataset.map(preprocess_data, batched=True) # We don't use as the model is already fine-tuned
# val_dataset = train_dataset.map(preprocess_data, batched=True) # We don't use as the model is already fine-tuned
test_dataset = test_dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/593 [00:00<?, ? examples/s]

In [21]:
test_dataset

Dataset({
    features: ['id', 'context', 'question', 'answers', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 593
})

In [22]:
def get_model_preds(model, dataset):
    preds = []
    for i in tqdm.tqdm(range(len(dataset))):  # Iterujemy po każdym elemencie
        input_ids = torch.tensor(dataset[i]["input_ids"]).unsqueeze(0).to('cpu')  # unsqueeze(0) dodaje wymiar batcha
        attention_mask = torch.tensor(dataset[i]["attention_mask"]).unsqueeze(0).to('cpu')  # unsqueeze(0) dodaje wymiar batcha
    
        # Generowanie odpowiedzi
        batch_generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=128,
        )
    
        preds.append(batch_generated_ids)
    
    # Dekodowanie wyników
    decoded_preds = []
    for g in preds:
        decoded_preds.append(tokenizer.decode(g[0], skip_special_tokens=True))
    return decoded_preds

In [23]:
exact_match = load("exact_match")

def compute_metrics(predictions, labels):
    # get exact match using buil-in funtion in eval package
    exac_m = exact_match.compute(predictions=predictions, references=labels)["exact_match"]
    
    encoded_preds = tokenizer(
        predictions,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    encoded_labels = tokenizer(
        labels,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    # Conversion needed to fit to sklearn API:
    predictions_flat = encoded_preds.flatten().tolist()  # Convert to list of integers
    labels_flat = encoded_labels.flatten().tolist()  # Convert to list of integers
    predictions_flat = np.array(predictions_flat, dtype=np.int32)
    labels_flat = np.array(labels_flat, dtype=np.int32)
    # compute f1 from sklearn API
    f1 = f1_score(labels_flat, predictions_flat, average='macro')

    return exac_m, f1

In [24]:
def eval_model_on_dataset(model, dataset):
    labels_text = [answer["text"][0] for answer in dataset["answers"]]
    
    model_preds = get_model_preds(model, dataset)
    em, f1 = compute_metrics(model_preds, labels_text)
    return em, f1

# Evaluation model on Val Dataset (Poquad)

In [25]:
#subset = test_dataset.select(range(20))
em, f1 = eval_model_on_dataset(model, test_dataset)
print(f"Exact Match: {em}, F1: {f1}")

100%|██████████| 593/593 [04:00<00:00,  2.47it/s]

Exact Match: 0.18043844856661045, F1: 0.13481808954883703


# Evaluation model on Test Dataset (Legal-Questions)

In [26]:
em, f1 = eval_model_on_dataset(model, val_dataset)
print(f"Exact Match: {em}, F1: {f1}")

100%|██████████| 46187/46187 [13:32:38<00:00,  1.06s/it]      


Exact Match: 0.6042176369974235, F1: 0.5613763916810257


In [49]:
def analyze_answer(model, dataset_record, data):
    pred = get_model_preds(model, dataset_record)
    print("Context:")
    print(data['context'])
    print("Question:")
    print(data['question'])
    print("True answer:")
    print(data['answers']['text'])
    print("predicted answer:")
    print(pred)

# Analyze of 10 examples from test dataset

In [50]:
analyze_answer(model, torch.utils.data.Subset(test_dataset, [3]), test_dataset[3])

100%|██████████| 1/1 [00:00<00:00,  1.74it/s]

Context:
Art. 35. 1. Wartość rzeczowych składników majątku obrotowego, które utraciły swoje cechy użytkowe lub przydatność, oraz odpadów ustala się nie później niż na dzień bilansowy w cenach sprzedaży netto możliwych do uzyskania. 2. Wartość produktów gotowych i towarów, z wyjątkiem towarów używanych znajdujących się w punktach sprzedaży oraz towarów przewidzianych do wieloletniej sprzedaży, zmniejsza się stopniowo, uwzględniając utratę ich wartości rynkowej, przez okres nie dłuższy niż 5 lat, poczynając od roku obrotowego następującego po roku, w którym je zakupiono lub wytworzono. 3. Odpisy aktualizujące: 1) wartość rzeczowych składników majątku obrotowego, o których mowa w ust. 1 i 2, oraz wynikające z wyceny według cen sprzedaży netto zamiast według cen nabycia (zakupu) lub kosztów wytworzenia - zwiększają pozostałe koszty operacyjne, 2) wartość udziałów w innych jednostkach oraz długoterminowych papierów wartościowych (lokat) - obciążają koszty operacji finansowych; jeżeli w wyni

Krótko, konkretnie i poprawnie, odpowiedz dobra.

In [51]:
analyze_answer(model, torch.utils.data.Subset(test_dataset, [4]), test_dataset[4])

100%|██████████| 1/1 [00:00<00:00,  1.57it/s]

Context:
Art. 74. 1. Armator, który wykonuje rybołówstwo morskie w polskich obszarach morskich, z naruszeniem przepisów ustawy, podlega karze pieniężnej do wysokości 1 000 000 złotych. 2. Karze, o której mowa w ust. 1, podlega również armator statku o polskiej przynależności, który wykonuje rybołówstwo morskie poza polskimi obszarami morskimi, z naruszeniem przepisów ustawy lub postanowień umów międzynarodowych, których Rzeczpospolita Polska jest stroną. 3. Minister właściwy do spraw rolnictwa określi, w drodze rozporządzenia, wysokość kar pieniężnych za naruszenia, o których mowa w ust. 1 i 2, zróżnicowane w zależności od ich rodzaju i społecznej szkodliwości.
Question:
Jakiej karze podlega armator, który wykonuje rybołówstwo morskie w polskich obszarach morskich, z naruszeniem przepisów ustawy?
True answer:
['Podlega karze pieniężnej do wysokości 1 000 000 złotych']
predicted answer:
['pieniężnej do wysokości 1 000 000 złotych']


Ponownie krótko i konkretnie, bardzo dobrze

In [52]:
analyze_answer(model, torch.utils.data.Subset(test_dataset, [14]), test_dataset[14])

100%|██████████| 1/1 [00:00<00:00,  2.49it/s]

Context:
Art. 29. 1. Funkcjonariusz podlega corocznemu opiniowaniu służbowemu 2. Przełożony funkcjonariusza może wydać opinię służbową o funkcjonariuszu także w innym terminie niż określony w ust. 1, jednakże nie wcześniej niż po upływie 6 miesięcy od wydania poprzedniej opinii. 3. W wydanej opinii służbowej przełożony zamieszcza ogólną ocenę o opiniowanym funkcjonariuszu wyrażoną w skali od 1 do 6 (niedostateczna, mierna, dostateczna, dobra, bardzo dobra, wzorowa). 4. Przełożony jest obowiązany zapoznać funkcjonariusza z wydaną o nim opinią służbową w ciągu 30 dni od dnia jej sporządzenia. 5. W przypadku, jeżeli niemożliwe jest zapoznanie funkcjonariusza z opinią służbową w terminie, o którym mowa w ust. 4, termin ten biegnie od dnia ustania przeszkody. 6. Funkcjonariuszowi przysługuje prawo wniesienia odwołania od wydanej o nim opinii służbowej do wyższego przełożonego w terminie 14 dni od dnia zapoznania się z opinią. 7. Od opinii wydanej przez Szefa BOR przysługuje funkcjonariuszow

Wiadomo o co chodzi, jednak odmiana słowa zła. Można ją zrozumieć jednak nie jest w odpowiedinej formie. Znowu dość krótko.

In [53]:
analyze_answer(model, torch.utils.data.Subset(test_dataset, [21]), test_dataset[21])

100%|██████████| 1/1 [00:00<00:00,  1.01it/s]

Context:
Art. 223. 1. Opłaty jednorazowe za zgłoszenia, wnioski, oświadczenia i inne czynności przewidziane w ustawie powinny być uiszczane z góry, o ile ustawa lub rozporządzenie, o którym mowa w art. 222 ust. 3, nie przewiduje uiszczenia opłaty na wezwanie Urzędu Patentowego w określonym terminie. 2. Opłata jednorazowa za zgłoszenie może być również uiszczona w ciągu jednego miesiąca od daty doręczenia wezwania Urzędu Patentowego. 3. Jeżeli w wyniku złożonego wniosku o ponowne rozpatrzenie sprawy decyzja lub postanowienie Urzędu Patentowego zostało uchylone, opłata uiszczona od tego wniosku podlega zwrotowi. 4. W razie nieuiszczenia w terminie opłaty, o której mowa w ust. 1, postępowanie wszczęte w wyniku dokonania zgłoszenia lub złożenia wniosku podlega umorzeniu, bądź czynność uzależniona od opłaty zostaje zaniechana.
Question:
Co się stanie jeżeli nie zostanie uiszczona opłata dla wniosku patentowego?
True answer:
['postępowanie wszczęte w wyniku dokonania zgłoszenia lub złożen

Bardzo dobrze, wręcz exact match

In [54]:
analyze_answer(model, torch.utils.data.Subset(test_dataset, [23]), test_dataset[23])


100%|██████████| 1/1 [00:00<00:00,  1.27it/s]

Context:
Art. 7. 1. W każdym tygodniu kierowca wykorzystuje odpoczynek w wymiarze co najmniej 45 kolejnych godzin. Odpoczynek może być skrócony nie więcej niż do 36 kolejnych godzin, jeżeli zostanie udzielony w miejscowości gdzie znajduje się siedziba pracodawcy lub w miejscu zamieszkania kierowcy i nie więcej niż do 24 kolejnych godzin, jeżeli zostanie udzielony w innym miejscu. Tygodniowy nieprzerwany odpoczynek obejmuje dobowy odpoczynek przypadający w dniu, w którym kierowca rozpoczął odpoczynek tygodniowy. 2. Okres odpoczynku niewykorzystany wskutek skrócenia, o którym mowa w ust. 1, kierowca powinien wykorzystać nie później niż przed upływem trzeciego tygodnia następującego po tygodniu, w którym odpoczynek został skrócony. 3. Tygodniowy odpoczynek kierowca powinien wykorzystać nie później niż po sześciu dobowych okresach prowadzenia pojazdu. 4. Przełożenie tygodniowego odpoczynku do końca szóstego dobowego okresu prowadzenia jest dopuszczalne w przypadku, gdy całkowity czas prowa

Tutaj gorzej, o temacie ale bez konkretów nic nie wynika z odpowiedzi niestety

In [55]:
analyze_answer(model, torch.utils.data.Subset(test_dataset, [33]), test_dataset[33])

100%|██████████| 1/1 [00:00<00:00,  1.28it/s]

Context:
Art. 41. 1. Kontroler jest obowiązany zachować w tajemnicy informacje, które uzyskał w związku z wykonywaniem czynności służbowych. 2. Obowiązek zachowania tajemnicy trwa również po ustaniu zatrudnienia.
Question:
Jakie obowiązki ma pracownik nadzorujący czynności kontrolne?
True answer:
['jest obowiązany zachować w tajemnicy informacje, które uzyskał w związku z wykonywaniem czynności służbowych']
predicted answer:
['zachowanie w tajemnicy informacji, które uzyskał w związku z wykonywaniem czynności służbowych']


Bardzo dobrze, popranie krótko i zwięźle.

In [56]:
analyze_answer(model, torch.utils.data.Subset(test_dataset, [57]), test_dataset[57])

100%|██████████| 1/1 [00:00<00:00,  1.99it/s]

Context:
Art. 107. 1. Krajowy Związek Kas zrzesza regionalne i branżowe kasy. 2. Siedzibą Krajowego Związku Kas jest Warszawa. 3. Krajowy Związek Kas prowadzi działalność na podstawie ustawy i statutu Krajowego Związku Kas.
Question:
Które miasto jest siedzibą Krajowego Związku Kas?
True answer:
['Warszawa']
predicted answer:
['Warszawa']


Dobrze, exact match, łatwy przykład

In [57]:
analyze_answer(model, torch.utils.data.Subset(test_dataset, [77]), test_dataset[77])

100%|██████████| 1/1 [00:00<00:00,  1.94it/s]

Context:
Art. 208. Oględzin lub badań ciała, które mogą wywołać uczucie wstydu, powinna dokonać osoba tej samej płci, chyba że łączą się z tym szczególne trudności; inne osoby odmiennej płci mogą być obecne tylko w razie konieczności.
Question:
Kto powinien dokonywać badań ciała, jeżeli może ono wywoływać uczucie wstydu?
True answer:
['Badań ciała, które mogą wywołać uczucie wstydu, powinna dokonać osoba tej samej płci, chyba że łączą się z tym szczególne trudności; inne osoby odmiennej płci mogą być obecne tylko w razie konieczności']
predicted answer:
['osoba tej samej płci']


Dużo krótsza, jednak konkretna i poprawna odpowiedz

In [58]:
analyze_answer(model, torch.utils.data.Subset(test_dataset, [113]), test_dataset[113])

100%|██████████| 1/1 [00:00<00:00,  2.94it/s]

Context:
Art. 8. § 1. W postępowaniu wykonawczym skazany może korzystać z pomocy obrońcy ustanowionego w tym postępowaniu lub pełnomocnika. § 2. W postępowaniu przed sądem skazany musi mieć obrońcę, jeżeli: 1) jest głuchy, niemy lub niewidomy, 2) zachodzi uzasadniona wątpliwość co do jego poczytalności, 3) nie ukończył 18 lat, 4) nie włada językiem polskim. § 3. Skazany pozbawiony wolności może porozumiewać się ze swoim obrońcą lub pełnomocnikiem będącym adwokatem podczas nieobecności innych osób oraz korespondencyjnie lub telefonicznie. Rozdział IV Postępowanie wykonawcze Oddział 1 Wykonywanie orzeczeń
Question:
Czy skazany który nie ukończył 18 lat musi mieć obrońcę?
True answer:
['tak']
predicted answer:
['nie']


Krótkoi konkretnie, porpawnie

In [60]:
analyze_answer(model, torch.utils.data.Subset(test_dataset, [257]), test_dataset[257])

100%|██████████| 1/1 [00:00<00:00,  1.08it/s]

Context:
Art. 3. Celem łowiectwa jest: 1) ochrona, zachowanie różnorodności i gospodarowanie populacjami zwierząt łownych, 2) ochrona i kształtowanie środowiska przyrodniczego na rzecz poprawy warunków bytowania zwierzyny, 3) uzyskiwanie możliwie wysokiej kondycji osobniczej i jakości trofeów oraz właściwej liczebności populacji poszczególnych gatunków zwierzyny przy zachowaniu równowagi środowiska przyrodniczego, 4) spełnianie potrzeb społecznych w zakresie uprawiania myślistwa, kultywowania tradycji oraz krzewienia etyki i kultury łowieckiej.
Question:
Co jest celem łowiectwa?
True answer:
['ochrona, zachowanie różnorodności i gospodarowanie populacjami zwierząt łownych']
predicted answer:
['ochrona, zachowanie różnorodności i gospodarowanie populacjami zwierząt łownych']


bardzo dobra odpowiedz, wręcz exact match

# Wnioski
Bardzo ciekawe i ponownie, wartościowe ćwiczenie. Szkoda niestety, że nie udało mi się mimo szczerych chęci dotrenować własnego modelu T5.
Próba była na macbooku M3 pro - estyomwany czas to było 700h - odpuściłem
Próba na Collabie - 10h - rozsądnie, jednak dwie próby collab zakończył, był to darmowy zasób ktory po prostu ktoś wywłaszczył. 3 próby już nie podjemowałem.


Wyniki bardzo ciekawe, tutaj warto zauważyć, że pomimo stosunkowo niskich metrych EM i F1, wyniki są dość dobre. Warto o tym pamiętać w Data Science, że metryki to jedno, jednak warto zawsze je przejrzeć własnoręcnzie i zinterprtować, podobna sytuacja ma miejsce w segmentacji obiektów, tam metryki też są zazwyczaj niskie, (mean-acc na poziomie 40%) jednak w prkatyce może to oznaczać bardzo dobre modele.
Praktycznie na 10 przetestowanych pytań model na 9 odpowiedział bardzo dobrze.
Jedno pomylił. Ma tendencje do udzielania krótszych odpowiedzi, co moim zdaniem na plus, ale zależy od problemu.

Zadanie pzowoliło znowu dotknąć praktycznego problemu, dotrenować na dużym popularnym zbiorze, i testować na 'prywatnym' datasecie. Pozwoliło mi to zrozumieć jak mogą działać, może nie najwieksze LLM jak Gpt-4 ale jakieś mniejsze które cieszą sie tez duża popularnościa. Ponadto zapoznanie się z bibliotekami datasets i transformers, na plus, ciężkie złożone biblioteki ale z ogromnymi możliowściami.
Bardzo ciekawe ćwiczenie. 

# Questions
- Does the performance on the validation dataset reflects the performance on your test set?
    Nie, do przewidzenia było że na zbiorze validacyjnym na kótrym był dotrenowany T5, wynik dużo lepszy, jednak mimo słabych metryk na Test secie, odpowiedzi modelu sensowe.
- What are the outcomes of the model on your test questions? Are they satisfying? If not, what might be the reason for that?
    Odpowiedzi są krótsze niż ground truem jednak moim zdaniem są ok, satysfakcjonujące.
- Why extractive question answering is not well suited for inflectional languages?
    Bo praktycznie zawsze odpowiedz wyekstraktowana będzie w złej formie, przez co wygląda to i brzmi nienaturalnie. W języku takim jak polski, dużo lepiej pasuje AQA niż EQA, które umożliwa zmianę te formy i odpowiedz bardziej w 'ludzkim' stylu.